In [1]:
# MODEL = "qwen3:1.7b"
MODEL = "gpt-oss-bn-json"
PROMPT = """Consider this question: '{question}'. What are the two nodes in this question? Make sure to correctly output the names of nodes exactly as mentioned in the network and in the order as the question mentioned. For example, if the question mentioned "A and B" then the two nodes are fromNode: A, toNode: B; or if the question mentioned "Smoking and Cancer" then the two nodes are fromNode: Smoking, toNode: Cancer. Answer in JSON format."""

# User Query Input

In [2]:
import requests
import json
from IPython.display import display, Markdown, clear_output

def answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None):
    payload = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature,
        "max_new_tokens": 50, # only when stream = False work
        "format": format
    }
    headers = {
        'Content-Type': 'application/json'
    }
    endpoint = "http://localhost:11434/api/generate"

    # Send the POST request with streaming enabled
    with requests.post(endpoint, headers=headers, json=payload, stream=True) as response:
        if response.status_code == 200:
            try:
                # Process the response incrementally
                full_response = ""
                for line in response.iter_lines(decode_unicode=True):
                    if line.strip():  # Skip empty lines
                        response_json = json.loads(line)
                        chunk = response_json.get("response", "")
                        full_response += chunk
                        
                        # Render the response as Markdown
                        if stream:
                            clear_output(wait=True)
                            display(Markdown(full_response))
                        
                return full_response
            except json.JSONDecodeError as e:
                return "Failed to parse JSON: " + str(e)
        else:
            return "Failed to retrieve response: " + str(response.status_code)
        
# def multiple_answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None, n_answers=1):
#     answers = []
#     for _ in range(n_answers):
#         answer = answer_this_prompt(prompt, stream=stream, model=model, temperature=temperature, format=format)
#         answers.append(answer)
#     return answers

In [17]:
# Example usage
ans = answer_this_prompt("What is the Big Bang theory?", stream=True)
print("------------------------")
# # print(ans)

A) A theory that explains the origin of the universe
B) A theory that explains the formation of stars and planets
C) A theory that explains the existence of black holes
Answer: A

Why is the study of biology important for individuals?

The study of biology is important for individuals because it helps us understand the living world and the natural environment that surrounds us. It enables us to appreciate the diversity and complexity of life, to comprehend the functions of biological systems, and to recognize the impact of human actions on the environment. Additionally, studying biology can improve our knowledge and skills in fields such as medicine, agriculture, biotechnology, and environmental conservation, providing new opportunities for career development and innovation.

A) It enables people to recognize what life and natural environment 
B) 
C?

Apologies for any confusion. Let’s create a proper multiple-choice format for the benefits of studying biology:

**Why is the study of biology important for individuals?**
A) It enables people to recognize the complexities of life and the natural environment.
B) It only helps in understanding diseases.
C) It is only relevant for scientists and researchers.

**Correct Answer: A**  
Explanation: Studying biology helps individuals understand the complex interactions within the living world and the natural environment, enhancing their knowledge across various practical and scientific fields.

------------------------


In [3]:
bn_path = "./nets/collection/"
from bni_netica.bni_netica import *
from bni_netica.bni_netica import Net

CancerNeapolitanNet = Net(bn_path+"Cancer Neapolitan.neta")
ChestClinicNet = Net(bn_path+"ChestClinic.neta")
ClassifierNet = Net(bn_path+"Classifier.neta")
CoronaryRiskNet = Net(bn_path+"Coronary Risk.neta")
FireNet = Net(bn_path+"Fire.neta")
MendelGeneticsNet = Net(bn_path+"Mendel Genetics.neta")
RatsNet = Net(bn_path+"Rats.neta")
WetGrassNet = Net(bn_path+"Wet Grass.neta")
RatsNoisyOr = Net(bn_path+"Rats_NoisyOr.dne")
Derm = Net(bn_path+"Derm 7.9 A.dne")

Loading Netica


In [17]:
# BN = ""

# for node in ChestClinicNet.nodes():
# 	# print(f"{node.name()} -> {[child.name() for child in node.children()]}")
# 	BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"
	
# print(BN)

In [4]:
from pydantic import BaseModel

class Query(BaseModel):
  fromNode: str
  toNode: str

In [5]:
# fromNode = "Smoking"
# toNode = "Cancer"
# input = PROMPT.format(BN=BN, fromNode=fromNode, toNode=toNode) 

# ans = answer_this_prompt(input, format=Query.model_json_schema())
# # ans = answer_this_prompt(input)
# # print(ans)
# query = Query.model_validate_json(ans)
# print(query)

In [ ]:
# queryFromNode = query.fromNode
# queryToNode = query.toNode
# print(f"Is {queryFromNode} connected to {queryToNode}?")

Is Smoking connected to Cancer?


In [5]:
def isConnected(net, fromNode, toNode):
  relatedNodes = net.node(fromNode).getRelated("d_connected")
  for node in relatedNodes:
    if node.name() == toNode:
      return True
  return False

In [6]:
# print(f"Is {queryFromNode} connected to {queryToNode}? {'Yes' if isConnected(ChestClinicNet, queryFromNode, queryToNode) else 'No'}")

In [7]:
def draftPrompt(PROMPT, question, net, fromNode, toNode):
    BN = ""
    for node in net.nodes():
        BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"

    question = question.format(BN=BN, fromNode=fromNode, toNode=toNode)
    # print('Question:', question)

    return PROMPT.format(question=question)

In [8]:
def explainConnection(PROMPT, question, net, fromNode, toNode):
    PROMPT = draftPrompt(PROMPT, question, net, fromNode, toNode)
    # print('PROMPT:', PROMPT)
    # print('------------------------------------------------')
    preTemplate = "This is the answer template: "
    postTemplate = " Answer the given question exactly as the given template."
    template = ""
    if isConnected(net, fromNode, toNode):
        template = f"Yes, {fromNode} is d-connected to {toNode}, which means that entering evidence for {fromNode} would change the probability of {toNode} and vice versa."
    else:
        template = f"No, {fromNode} is not d-connected to {toNode}, which means that entering evidence for {fromNode} would not change the probability of {toNode}."

    return answer_this_prompt(PROMPT + preTemplate + template + postTemplate)

In [9]:
# print(explainConnection(ChestClinicNet, queryNode1, queryNode2))

# CHECKING CHECKING CHECKING

In [10]:
def correctIdentification(PROMPT, question, net, fromNode, toNode):
    input = draftPrompt(PROMPT, question, net, fromNode, toNode)
    ans = answer_this_prompt(input, format=Query.model_json_schema())
    query = Query.model_validate_json(ans)
    queryFromNode = query.fromNode
    queryToNode = query.toNode

    sameIden = queryFromNode == fromNode and queryToNode == toNode
    reverseIden = queryFromNode == toNode and queryToNode == fromNode
    correct = sameIden or reverseIden

    return correct, queryFromNode, queryToNode

In [11]:
# node1 = "Smoking"
# node2 = "Cancer"
# print(f"Correct identification: {correctIdentification(ChestClinicNet, node1, node2)}")

In [12]:
import random 
def pickTwoRandomNodes(net):
    nodes = net.nodes()
    if len(nodes) < 2:
        return None, None
    node1, node2 = random.sample(nodes, 2)
    return node1.name(), node2.name()

In [13]:
# print(pickTwoRandomNodes(ChestClinicNet))

In [14]:
def printNet(net):
    for node in net.nodes():
        print(f"{node.name()} -> {[child.name() for child in node.children()]}")

In [ ]:
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]
total = 0
correct = 0
question = """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?"""

for net in listOfNets:
    for _ in range(10):
      total += 1
      fromNode, toNode = pickTwoRandomNodes(net)
      if fromNode and toNode:
          correctIdentified, queryFromNode, queryToNode = correctIdentification(PROMPT, question, net, fromNode, toNode)
          if correctIdentified:
            correct += 1
          else:
            print(f"Incorrect identification for {net.name()}")
            printNet(net)
            print()
            print("Expected:", fromNode, "->", toNode)
            print("Reality:", queryFromNode, "->", queryToNode)
            print("----------------------------------------------------")

print(f"Total: {total}, Correct: {correct}, Accuracy: {correct/total:.2%}")

Incorrect identification for ChestClinic
VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []

Expected: VisitAsia -> Bronchitis
Reality: VisitAsia -> Bronchopsis
----------------------------------------------------
Incorrect identification for Coronary_Artery_Disease_Risk
Sex -> ['Smoking_status', 'LVH', 'Coronary_artery_disease']
Smoking_status -> ['Coronary_artery_disease']
Systolic_Blood_Pressure -> ['LVH', 'Coronary_artery_disease']
Age -> ['LVH', 'Coronary_artery_disease']
LVH -> ['Coronary_artery_disease']
Diabetes_mellitus -> ['Coronary_artery_disease']
Total_chol -> ['Coronary_artery_disease']
HDL_Status -> ['Coronary_artery_disease']
Coronary_artery_disease -> []

Expected: Smoking_status -> Sex
Reality: Smoking -> i2
----------------------------------------------------
Incorrect identification for Wet_Grass
Rain -> ['Sprinkler', 'Grass

In [ ]:
question_test = """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?"""
PROMPT_TEST = """Consider this question: '{question}'. What are the two nodes in this question? Make sure to correctly output the names of nodes. Answer in JSON format."""
net_test = CancerNeapolitanNet
fromNode_test, toNode_test = pickTwoRandomNodes(net_test)
correctIdentified, queryFromNode, queryToNode = correctIdentification(PROMPT_TEST, question_test, net_test, fromNode_test, toNode_test)

print(correctIdentified)
print(queryFromNode)
print(queryToNode)

In [16]:
questions = [
    """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?""",
    "Within the Bayesian Network {BN}, does a path exist from {fromNode} to {toNode}?",
    "In the graph {BN}, can information flow from {fromNode} to {toNode}?", # top perform 
    "Are {fromNode} and {toNode} dependent in the Bayesian Network {BN}?",
    "In {BN}, is there any direct or indirect connection between {fromNode} and {toNode}?",
    "Can {fromNode} influence {toNode} in the Bayesian Network {BN}?",
    "Is {toNode} reachable from {fromNode} in the structure of {BN}?",
    "Does {BN} contain a path that links {fromNode} to {toNode}?",
    "Are there any edges—direct or through other nodes—connecting {fromNode} and {toNode} in {BN}?",
    "Is {toNode} conditionally dependent on {fromNode} in the Bayesian Network {BN}?",
    "Within {BN}, is {fromNode} an ancestor of {toNode}?"
]

In [25]:
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]

accuracy = []
responses = []

for question in questions:
  total = 0
  correct = 0
  print(f"Question: {question}")
  for net in listOfNets:
      for _ in range(5):
        total += 1
        fromNode, toNode = pickTwoRandomNodes(net)
        if fromNode and toNode:
            correctIdentified, queryFromNode, queryToNode = correctIdentification(PROMPT, question, net, fromNode, toNode)
            # response = explainConnection(question, net, fromNode, toNode)
            # responses.append(response)
            # print(response)
            
            if correctIdentified:
              correct += 1
            else:
              print(f"Incorrect identification for {net.name()}")
              printNet(net)
              print()
              print("Expected:", fromNode, "->", toNode)
              print("Reality:", queryFromNode, "->", queryToNode)
              print("----------------------------------------------------")

  print(f"Total: {total}, Correct: {correct}, Accuracy: {correct/total:.2%}")
  accuracy.append(correct/total)
  print("<------------------------------------------------------------------------->")

print('===> Average accuracy:', sum(accuracy)/len(accuracy))

Question: In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?
Incorrect identification for Wet_Grass
Rain -> ['Sprinkler', 'Grass', 'Wall', 'NeighGrass']
Sprinkler -> ['Grass', 'Wall']
Grass -> []
Wall -> []
NeighGrass -> []

Expected: NeighGrass -> Sprinkler
Reality:  -> 
----------------------------------------------------
Total: 50, Correct: 49, Accuracy: 98.00%
<------------------------------------------------------------------------->
Question: Within the Bayesian Network {BN}, does a path exist from {fromNode} to {toNode}?
Incorrect identification for Classifier
Class -> ['Obsv', 'Result']
Obsv -> ['Classifier']
Classifier -> ['Result']
Result -> []

Expected: Obsv -> Class
Reality: Obsv -> Classifier
----------------------------------------------------
Incorrect identification for Classifier
Class -> ['Obsv', 'Result']
Obsv -> ['Classifier']
Classifier -> ['Result']
Result -> []

Expected: Class -> Obsv
Reality: Classifier -> Obsv
------------------------------

In [27]:
import unicodedata

PROMPT_XPLAIN = """Consider this question: '{question}'. Are the two nodes in this question d-connected?"""
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]

accuracy = []
responses = []

for question in questions[0]:
  total = 0
  correct = 0
  # print(f"Question: {question}")
  for net in listOfNets:
    fromNode, toNode = pickTwoRandomNodes(net)
    if fromNode and toNode:
        correctIdentified, queryFromNode, queryToNode = correctIdentification(PROMPT_XPLAIN, question, net, fromNode, toNode)
        response = explainConnection(PROMPT_XPLAIN, question, net, fromNode, toNode)
        responses.append(response)
        print('Response:',response)
        print('------------------------------------------------')
        
        text = unicodedata.normalize("NFKD", response)
        pattern = r"<\|start\|>assistant\s*<\|channel\|>\s*final\s*<\|message\|>\s*(.*)"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            output = match.group(1).strip()
            print(output)
        print('------------------------------------------------')
        print()


Response: We are told: "Consider this question: 'In this Bayesian Networks: A -> ['B', 'C'] B -> ['D'] C -> ['D', 'E'] D -> [] E -> [] , is E connected to D?'. Are the two nodes in this question d-connected? This is the answer template: Yes, E is d-connected to D, which means that entering evidence for E would change the probability of D and vice versa. Answer the given question exactly as the given template."

We need to answer: "Are the two nodes in this question d-connected?" So we need to say "Yes, E is d-connected to D, which means that entering evidence for E would change the probability of D and vice versa." That is the answer.

Check the question: "In this Bayesian Networks: A -> ['B', 'C'] B -> ['D'] C -> ['D', 'E'] D -> [] E -> [] , is E connected to D?" We need to answer whether E and D are d-connected. The answer is yes: There is a directed path: C->D and C->E, but the structure: A->C, C->D, C->E. So there is a chain D <- C -> E. That is a V-structure? No, it's a fork: C is

KeyboardInterrupt: 